In [189]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb

%matplotlib inline
plt.rcParams['figure.figsize'] = (30, 50)
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn import preprocessing
from sklearn.externals import joblib


C:\Users\Jose Manuel\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [190]:
url = f"C:\\Users\\Jose Manuel\\Desktop\\Ingenieria Informatica\\BigData\\Proyecto\\CLO_YouTube\\spark\\dataStreaming1000rows.csv"

df = pd.read_csv(url,sep=';')
# Suponemos que un video triunfa si tiene más visitas que la media de videos en tendencia
# por lo que las columnas que tiene relación con las visitas hay que eliminarlas.

#df["Tags"]= df["Tags"].str.split(",",1)
dftags=df.Tags.str.split(",",5,expand=True)
df = pd.merge(df,dftags, right_index=True, left_index=True)
df = df.drop(columns=['Tags'])
df = df.rename(columns={0: 'Tag0',1: 'Tag1',2: 'Tag2',3: 'Tag3',4: 'Tag4',5: 'Tag5'})
df['test'] = np.where(df['View_count']>df['View_count'].mean(), 1, 0)
df = df.drop(columns=['Channel_subscribers','Video_title','Description', 'View_count','Like_count', 'Dislike_count','Like_count', 'Comment_count','Favorite_count'])
df = df.replace({float('NaN'):'0'})
df.head()
#df=df+dftags
#dftags = dftags.drop(columns=['5'])

,videoID,Published_at,Channel_title,Tag0,Tag1,Tag2,Tag3,Tag4,Tag5,test
0,5hkPDgf9qJI,2019-12-11T11:00:02.000Z,Thisiz Balti,Balti,denia,hamouda,حمودة,ya lili,"دنيا,2019,new,rap,بلطي,rap tunisien,tunisie,tu...",0
1,lTK-m7178-4,2019-12-11T07:13:01.000Z,مطبخ وتدابير زهرة,0,0,0,0,0,0,0
2,r_fWTgHB4Jg,2019-12-11T07:52:02.000Z,Dadah Cuisine مطبخ و تدابير دداح,0,0,0,0,0,0,0
3,ZF_fmBAM60I,2019-12-11T08:23:04.000Z,cuisine loudjain dz,0,0,0,0,0,0,0
4,oi-XeCez5Hc,2019-12-11T10:00:05.000Z,Saud Brothers,تحدي,تحديات,الكنز,تحدي الكنز,البحث,"البحث عن الكنز,تحدي البحث عن الكنز,رحلة,رحلة ا...",0


In [191]:
# Encoding the variable

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                print (col)
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                print (colname)
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
    
df_encoded = MultiColumnLabelEncoder(columns=['videoID','Published_at','Channel_title','Tag0','Tag1','Tag2','Tag3','Tag4','Tag5']).fit_transform(df)
df_encoded.head()

videoID
Published_at
Channel_title
Tag0
Tag1
Tag2
Tag3
Tag4
Tag5


,videoID,Published_at,Channel_title,Tag0,Tag1,Tag2,Tag3,Tag4,Tag5,test
0,105,925,684,51,321,372,736,597,752,0
1,741,914,917,3,1,8,5,5,2,0
2,832,916,162,3,1,8,5,5,2,0
3,550,917,768,3,1,8,5,5,2,0
4,787,919,600,777,765,740,724,704,717,0


In [192]:
array = df_encoded.values
X = array[:,0:9]
Y = array[:,9]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

In [193]:
# Fit the model on training set
model = LogisticRegression()
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

C:\Users\Jose Manuel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8027777777777778

In [194]:
# save the model to disk
filename = 'ml_model.sav'
joblib.dump(model, filename)
 
# some time later...
 


['ml_model.sav']

In [195]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)

0.8027777777777778
